In [60]:
# inladen benodigde packages
import pandas as pd
import numpy as np

In [103]:
def merge_annotated_data(naam_dataset, n_annotators, n_annotations = 3):
    """ functie die de gesplitste data gesplitst door split_annotated_data weer kan samenvoegen. 
    De datasets met de naam naam_dataset_n_annotators.csv worden ingeladen. Vervolgens wordt de oorspronkelijke dataset gereconstrueerd
    uit de ingeladen data. De oorspronkelijke zinnen en het overzicht van de labels wordt gereturned.
    
    Input:
    naam_dataset: str, de naam van de documenen die de annotators hebben ingevuld
    n_annotators: int, het aantal annotators
    n_annotations: int, het aantal annotations per zin. default 3
    
    Output:
    merged_data: een DataFrame met daarin de oorspronkelijke data en de n_annotations labels die zijn toegekend.
    """
    # inladen path naar data en creëren lege frames
    dataset_path = "~/share/Milena/programma-discriminatie-en-racisme/01_datapreparatie/"
    merged_data = pd.DataFrame()
    
    for i in range(n_annotators):
        # inladen annotated documenten en samenvoegen
        annotated_doc = pd.read_csv(dataset_path + naam_dataset + "_" + str(i) + '.csv', sep = ',')
        columns = list(annotated_doc.columns)
        
        # document opstellen
        if i == 0:
            merged_data = annotated_doc
        else:
            merged_data = pd.concat([merged_data, annotated_doc])
            
    merged_data = merged_data.groupby(['Unnamed: 0']).agg({'id': 'last', 'zin': 'last', 'score': 'mean'})
    merged_data = merged_data.sort_values(by = ['Unnamed: 0'])
    
    return pd.DataFrame(merged_data)

merged = merge_annotated_data('te_labellen', n_annotators = 5, n_annotations = 3)
display(merged)

,id,zin,score
Unnamed: 0,,,
0,d4cfb963-de8d-400e-bb87-5493b794a255_1_4adc737...,...,NaN
1,d4cfb963-de8d-400e-bb87-5493b794a255_2_4adc737...,Onze referentie\nVerdel...,NaN
2,7eb69333-4557-4594-b5be-1276e3c0f2da_2_027bdd4...,"groeifaciliteit voor de 23987, nr. aanspraa...",NaN
3,ad741e24-86a7-47a2-ad12-fbde149fe4bc_8_e4269c3...,hoofdstuk 01 | Inleiding ...,NaN
4,ad741e24-86a7-47a2-ad12-fbde149fe4bc_17_e4269c...,"hoofdstuk 03 | Effectiviteit, resultaten, nade...",NaN
...,...,...,...
6328,490faff2-b50c-4a78-afdc-f36ed34ed3df_10_039b80...,QUICK SCAN D....,NaN
6329,490faff2-b50c-4a78-afdc-f36ed34ed3df_11_039b80...,QUICK SCAN...,NaN
6330,490faff2-b50c-4a78-afdc-f36ed34ed3df_15_039b80...,QUICK SCA...,NaN


In [50]:
# output checken
print(list(merged[0]))

[0, 0, 'Beslisnota bij Kamerbrief over Geografische Balans Subsidieregeling en Beleidsintensiveringen Slavernijverleden', '                                                                                    TER BESLUITVORMING\n                                                                                    Nota actief openbaar\n                                                                                    Ja\n                                                                                    Onze referentie\nAan                      StasBZK en MBZK                                            2024-0000015654\nVan                      Programma <mark style="background: #00ced1!important">slavernijverleden</mark>                                Datum\n                                                                                    9 januari 2024\n                                                                                    Opgesteld door\n                          (0) Nota 

In [91]:
### OUDE CODE
# merge annotated data
def merge_annotated_data(naam_dataset, n_annotators, n_annotations):
    """ functie die de gesplitste data gesplitst door split_annotated_data weer kan samenvoegen. 
    De datasets met de naam naam_dataset_n_annotators.csv worden ingeladen. Vervolgens wordt de oorspronkelijke dataset gereconstrueerd
    uit de ingeladen data. De oorspronkelijke zinnen en het overzicht van de labels wordt gereturned.
    
    Input:
    naam_dataset: str, de naam van de documenen die de annotators hebben ingevuld
    n_annotators: int, het aantal annotators
    n_annotations: int, het aantal annotations per zin
    
    Output:
    merged_data: een DataFrame met daarin de oorspronkelijke data en de n_annotations labels die zijn toegekend.
    """
    # inladen path naar data en creëren lege frames
    dataset_path = "~/share/Milena/programma-discriminatie-en-racisme/01_datapreparatie/"
    merged_data = []
    to_merge = []
    lengths = []
    
    for i in range(n_annotators):
        # inladen annotated documenten en samenvoegen
        annotated_doc = pd.read_csv(dataset_path + naam_dataset + "_" + str(i) + '.csv', sep = ',')
        lengte = len(annotated_doc)
        to_merge.append(annotated_doc)
        lengths.append(lengte)
    
    # oorspronkelijke lengte van document terugberekenen
    oorspronkelijke_lengte = int(sum(lengths) / 3) # dit zou altijd een geheel getal moeten worden
    #print(oorspronkelijke_lengte)
    #print(len(to_merge))
    
    # met de counter variabele tel je hoe vaak een zin al is toegevoegd (maximaal n_annotations keer)
    counter = 0
    max_counter = n_annotations - 1
    
    for i in range(oorspronkelijke_lengte):
        for list_counter in range(n_annotators): # op deze manier lopen we de lijsten op volgorde door
            # voor als lijsten ongelijke lengte hebben: de loop moet geëxit worden als er geen items meer zijn om toe te voegen
            if i == len(to_merge[list_counter]): 
                break
            # voor het eind van de lijst indexeren we het item
            else:
                item = to_merge[list_counter].iloc[i]
                if counter == 0: # eerste keer dat we de zin tegenkomen, voegen we die geheel toe
                    counter += 1
                    merged_data.append(item)
                elif counter < max_counter: # daarna voegen we alleen het laatste item (de score) toe
                    counter += 1
                    merged_data.append(item[-1])
                else: # voegen we de score toe en resetten we de counter
                    counter = 0
                    merged_data.append(item[-1])
        else:
            continue
        break
    return pd.DataFrame(merged_data)

merged = merge_annotated_data('te_labellen', n_annotators = 5, n_annotations = 3)

TypeError: object of type 'numpy.int64' has no len()